In [42]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from copy import deepcopy
import math

df = pd.read_csv('data/mobile.csv', index_col=0)
print(len(df))
df.head()

400


,battery_power,clock_speed,four_g,int_memory,n_cores,pc,ram,talk_time,touch_screen,wifi,price_range
0,1142,1.4,0,6,8,8,663,5,1,1,0
1,728,1.7,1,5,2,20,834,5,1,0,0
2,1868,0.5,1,40,8,17,298,17,1,0,0
3,890,2.2,0,44,8,13,751,3,0,0,0
4,1433,1.6,0,4,8,11,258,4,1,1,0


In [43]:
cols = ['battery_power', 'clock_speed', 'talk_time']

df = df[cols]
df.head()

,battery_power,clock_speed,talk_time
0,1142,1.4,5
1,728,1.7,5
2,1868,0.5,17
3,890,2.2,3
4,1433,1.6,4


## ranking standartization

In [44]:
diff = df.max() - df.min()
diff

battery_power    1495.0
clock_speed         2.5
talk_time          18.0
dtype: float64

In [45]:
df_norm = (df - df.min()).divide(diff)
df_norm.head()

,battery_power,clock_speed,talk_time
0,0.428094,0.36,0.166667
1,0.151171,0.48,0.166667
2,0.913712,0.00,0.833333
3,0.259532,0.68,0.055556
4,0.622742,0.44,0.111111


In [52]:
data_scatter = df_norm.multiply(df_norm).values.sum()
data_scatter

377.47480466092503

In [77]:
z, mu, c = np.linalg.svd(df_norm)
mu

array([17.20123172,  6.7003017 ,  6.05791955])

## Contribution of principal components:

In [61]:
np.multiply(mu.T, mu)

array([295.88237256,  44.89404281,  36.69838929])

### in percent:

In [78]:
np.multiply(mu.T, mu) / data_scatter * 100

array([78.38466804, 11.89325546,  9.7220765 ])

## hidden factor

In [81]:
z = -z
c = -c
c[0,:]

array([0.59979566, 0.50652763, 0.61941499])

$$100=(0.599 * 100 + 0.506 * 100 + 0.619 * 100) * \alpha$$
$$ \alpha = 0.58$$

$$ Z = 0.347 * \text{batttery} + 0.293 * \text{clock} + 0.359 * \text{talk}$$